In [4]:
# import huggingface transformers
# import numpy as np
# import pandas as pd
# import torch
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
from datasets import Dataset

In [7]:
# Dataset from jsonl
ds = Dataset.from_json('data/train_with-reference.jsonl')

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 204.03it/s]
                                                        

Dataset json downloaded and prepared to /Users/amasand/.cache/huggingface/datasets/json/default-75643ea6b2d98652/0.0.0. Subsequent calls will reuse this data.


In [15]:
ds[0]['utterances']

[{'text': 'A) pull through', 'speaker': 'student'},
 {'text': 'OK great', 'speaker': 'teacher'},
 {'text': 'Not sure about the meaning of the second one... Does that person mean that being the prime minister he had to survive??',
  'speaker': 'student'}]

In [16]:
ds[0]['response']

{'text': 'Ah yes good question - this is a bit ambiguous....',
 'speaker': 'teacher'}

In [28]:
# load jsonl 
import os 
import json
import pandas as pd

data = {
    "train": [],
    "dev": [],
}

dataset_names = {
    "train": "train_with-reference",
    "dev": "dev_without-reference"
}
for split in dataset_names:
    with open(os.path.join("data", "raw", f"{dataset_names[split]}.jsonl"), "r") as f:
        for line in f:
            data_line = json.loads(line)
            sample = {"context": "", "response": ""}
            utterances = data_line["utterances"]
            # [{'text': 'A) pull through', 'speaker': 'student'}, {'text': 'OK great', 'speaker': 'teacher'}, {'text': 'Not sure about the meaning of the second one... Does that person mean that being the prime minister he had to survive??', 'speaker': 'student'}] {'text': 'Ah yes good question - this is a bit ambiguous....', 'speaker': 'teacher'}
            sample["context"] = "\n".join([f"{x['speaker']}: {x['text']}" for x in utterances])
            if "response" in data_line.keys():
                response = data_line["response"]
                sample["response"] = response['speaker'] + ": " + response['text']
            print (sample)
            data[split].append(sample)

{'context': 'student: A) pull through\nteacher: OK great\nstudent: Not sure about the meaning of the second one... Does that person mean that being the prime minister he had to survive??', 'response': 'teacher: Ah yes good question - this is a bit ambiguous....'}
{'context': "student: willpower?)\nteacher: Yes!\nteacher: So, a bit of willpower, and kids will be super successful!\nteacher: Do you think it worked?\nstudent: yes\nteacher: No! :))\nstudent: why?\nteacher: because human beings are way more complex than that\nteacher: You know how in science it's very important that the results of a study be replicated?\nstudent: yes, it's very important", 'response': 'teacher: yes, so scientists did more and more marshmallow tests and followed the lives of the kids, and they did not get the same results as the original study'}
{'context': "student: I see. I think it means 'don't make me start'\nteacher: Yes actually - especially something like laughing or in the example i just sent?\nteache

In [38]:
train_ds = Dataset.from_list(data['train'])
train_ds.save_to_disk('data/train_with-reference')

dev_ds = Dataset.from_list(data['dev'])
dev_ds.save_to_disk('data/dev_without-reference')